In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


df = pd.read_csv("tmdb_5000_movies.csv", engine="python")
df = df[["genres", "original_title",]].dropna().reset_index(drop=True)
df = df[:5500]


df['genres'] = df['genres'].str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.replace("'", "", regex=False).str.replace(',', ' ', regex=False)
df['genres'] = df['genres'].str.lower()

tfidf = TfidfVectorizer(stop_words='english')


tfidf_matrix = tfidf.fit_transform(df['genres'])


similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend_by_cat(cat, num_recommendations=10):

    filtered_df = df[df['genres'].str.contains(cat.lower(), case=False)]
    
    if filtered_df.empty:
        return f"No movie found '{cat}'."
    

    recommended_product = []

    for idx in filtered_df.index:

        sim_scores = list(enumerate(similarity_matrix[idx]))
        

        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        

        sim_scores = sim_scores[1:num_recommendations + 1]  
        

        product_indices = [i[0] for i in sim_scores]
        pro = df['original_title'].iloc[product_indices].tolist()
        recommended_product.extend(pro)
    

    recommended_product = list(set(recommended_product))
    
    return recommended_product[:num_recommendations]


recommended_product = recommend_by_cat("Action", num_recommendations=10)
print("Recommended movies:", recommended_product)

Recommended movies: ['U-571', 'John Carter', 'The Man from U.N.C.L.E.', 'Universal Soldier: The Return', 'The Man in the Iron Mask', 'La niña santa', 'Walk Hard: The Dewey Cox Story', 'Star Trek: The Motion Picture', 'Resident Evil: Retribution', 'Avengers: Age of Ultron']
